In [2]:
# Dataprep for Gaussian Blur Detect
from PIL import ImageFile, ImageFilter
ImageFile.LOAD_TRUNCATED_IMAGES = True
import shutil
import pandas as pd
import os

def blur_image(image):
    # Apply Gaussian blur directly on the PIL image object
    return image.filter(ImageFilter.GaussianBlur)

In [3]:
from datasets import load_dataset
dataset = load_dataset("yh0701/FracAtlas_dataset", trust_remote_code=True)

C:\Users\LEGION\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
dataset['test'][0]

{'image_id': 'IMG0001287.jpg',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=373x454>,
 'hand': 0,
 'leg': 1,
 'hip': 0,
 'shoulder': 0,
 'mixed': 0,
 'hardware': 0,
 'multiscan': 0,
 'fractured': 0,
 'fracture_count': 0,
 'frontal': 1,
 'lateral': 0,
 'oblique': 0,
 'localization_metadata': {'width': 373, 'height': 454, 'depth': 3},
 'segmentation_metadata': None}

In [7]:
def create_directories(directory_paths):
    for path in directory_paths.values():
        os.makedirs(path, exist_ok=True)

In [4]:
for j in ['test', 'validation', 'train']:
    for i in range(len(dataset[j])):
        pil_image = dataset[j][i]['image']
        blurred_image = blur_image(pil_image)
        image_name = dataset[j][i]['image_id']
        blurred_image.save(f'Blured_Images\{image_name}')

In [5]:
# Distribute Blured Images for Detection Training
train_csv = "FracAtlas/FracAtlas/Utilities/Fracture Split/train.csv"
valid_csv = "FracAtlas/FracAtlas/Utilities/Fracture Split/valid.csv"
test_csv = "FracAtlas/FracAtlas/Utilities/Fracture Split/test.csv"

source_dir = "Blured_Images"

dest_dirs = {
    'train': "Blured_Training_Detection/dataset/train/images",
    'valid': "Blured_Training_Detection/dataset/valid/images",
    'test': "Blured_Training_Detection/dataset/test/images", 
}

def copy_images(csv_file, target_dir):
    df = pd.read_csv(csv_file)
    os.makedirs(target_dir, exist_ok=True)
    for image_name in df['image_id']:
        src_path = os.path.join(source_dir, image_name)
        dst_path = os.path.join(target_dir, image_name)
        shutil.copy(src_path, dst_path)

copy_images(train_csv, dest_dirs['train'])
copy_images(valid_csv, dest_dirs['valid'])
copy_images(test_csv, dest_dirs['test'])

In [6]:
# Distribute Labels for Detection Training
source_dir = "FracAtlas/FracAtlas/Annotations/YOLO"

dest_dirs = {
    'train': "Blured_Training_Detection/dataset/train/labels",
    'valid': "Blured_Training_Detection/dataset/valid/labels",
    'test': "Blured_Training_Detection/dataset/test/labels", 
}

def copy_labels(csv_file, target_dir):
    df = pd.read_csv(csv_file)
    for image_name in df['image_id']:
        label_name = image_name.replace('.jpg', '.txt')
        shutil.copy(os.path.join(source_dir, label_name), os.path.join(target_dir, label_name))

copy_labels(train_csv, dest_dirs['train'])
copy_labels(valid_csv, dest_dirs['valid'])
copy_labels(test_csv, dest_dirs['test'])

In [8]:
# Dataprep for Gaussian Blur Segmentation
source_dir = "Blured_Images"

dest_dirs = {
    'train': "Blured_Training_Segmentation/dataset/train/images",
    'valid': "Blured_Training_Segmentation/dataset/valid/images",
    'test': "Blured_Training_Segmentation/dataset/test/images", 
}
create_directories(dest_dirs)

def copy_images(csv_file, target_dir):
    df = pd.read_csv(csv_file)
    os.makedirs(target_dir, exist_ok=True)
    for image_name in df['image_id']:
        src_path = os.path.join(source_dir, image_name)
        dst_path = os.path.join(target_dir, image_name)
        shutil.copy(src_path, dst_path)

copy_images(train_csv, dest_dirs['train'])
copy_images(valid_csv, dest_dirs['valid'])
copy_images(test_csv, dest_dirs['test'])

In [9]:
# Distribute Labels for Segmentation Training
source_dir = "YOLO_Segmentation_Labels"

dest_dirs = {
    'train': "Blured_Training_Segmentation/dataset/train/labels",
    'valid': "Blured_Training_Segmentation/dataset/valid/labels",
    'test': "Blured_Training_Segmentation/dataset/test/labels", 
}
create_directories(dest_dirs)

def copy_labels(csv_file, target_dir):
    df = pd.read_csv(csv_file)

    for image_name in df['image_id']:
        label_name = image_name.replace('.jpg', '.txt')
        shutil.copy(os.path.join(source_dir, label_name), os.path.join(target_dir, label_name))

copy_labels(train_csv, dest_dirs['train'])
copy_labels(valid_csv, dest_dirs['valid'])
copy_labels(test_csv, dest_dirs['test'])